### 

In [1]:
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable


In [2]:
pip install transformers -U


Defaulting to user installation because normal site-packages is not writeable
  Using cached transformers-4.40.1-py3-none-any.whl (9.0 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.36.2
    Uninstalling transformers-4.36.2:
      Successfully uninstalled transformers-4.36.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy-transformers 1.3.4 requires transformers<4.37.0,>=3.4.0, but you have transformers 4.40.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tf-keras


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 10.6 MB/s eta 0:00:00m eta 0:00:010:01:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
from datasets import load_dataset
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

# Load dataset
dataset = load_dataset("surrey-nlp/PLOD-CW")

# Extract features and labels
train_data = dataset["train"]
val_data = dataset["validation"]
test_data = dataset["test"]

X_train_data = train_data["tokens"]
y_train_data = train_data["ner_tags"]

X_val_data = val_data["tokens"]
y_val_data = val_data["ner_tags"]

X_test_data = test_data["tokens"]
y_test_data = test_data["ner_tags"]

# Convert multi-labels to binary format
mlb = MultiLabelBinarizer()
y_train_binarized = mlb.fit_transform(y_train_data)
y_val_binarized = mlb.transform(y_val_data)
y_test_binarized = mlb.transform(y_test_data)

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Convert lists of tokens back to strings
X_train_data = [" ".join(tokens) for tokens in train_data['tokens']]
X_val_data = [" ".join(tokens) for tokens in val_data['tokens']]
X_test_data = [" ".join(tokens) for tokens in test_data['tokens']]

# Tokenize text data
train_encodings = tokenizer(X_train_data, truncation=True, padding='max_length', max_length=128, return_tensors="tf")
val_encodings = tokenizer(X_val_data, truncation=True, padding='max_length', max_length=128, return_tensors="tf")
test_encodings = tokenizer(X_test_data, truncation=True, padding='max_length', max_length=128, return_tensors="tf")

# Load BERT model
bert_model = TFBertModel.from_pretrained("bert-base-uncased")

# Define the neural network model
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name="attention_mask")

bert_output = bert_model(input_ids, attention_mask=attention_mask)
pooled_output = bert_output.pooler_output  
x = tf.keras.layers.Dense(128, activation='relu')(pooled_output)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(y_train_binarized.shape[1], activation='sigmoid')(x)

model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    {'input_ids': train_encodings['input_ids'], 'attention_mask': train_encodings['attention_mask']},
    y_train_binarized,
    validation_data=(
        {'input_ids': val_encodings['input_ids'], 'attention_mask': val_encodings['attention_mask']},
        y_val_binarized
    ),
    epochs=10,
    batch_size=32,
    verbose=1
)

# Evaluate the model on the test set
test_scores = model.evaluate(
    {'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']},
    y_test_binarized,
    verbose=0
)
print("Test Loss:", test_scores[0])
print("Test Accuracy:", test_scores[1])

# Predict labels for the test set
y_pred_proba = model.predict({'input_ids': test_encodings['input_ids'], 'attention_mask': test_encodings['attention_mask']})
y_pred_binary = (y_pred_proba > 0.5).astype(int)



2024-04-26 07:11:48.328768: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-26 07:11:51.006216: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-04-26 07:12:04.606593: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-26 07:12:04.635959: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Fo

Epoch 1/10
34/34 [==============================] - 279s 7s/step - loss: 0.3858 - accuracy: 0.0951 - val_loss: 0.3739 - val_accuracy: 0.0794
Epoch 2/10
34/34 [==============================] - 250s 7s/step - loss: 0.3636 - accuracy: 0.1017 - val_loss: 0.3795 - val_accuracy: 0.0794
Epoch 3/10
34/34 [==============================] - 384s 11s/step - loss: 0.3663 - accuracy: 0.1017 - val_loss: 0.3532 - val_accuracy: 0.0794
Epoch 4/10
34/34 [==============================] - 426s 13s/step - loss: 0.3580 - accuracy: 0.1017 - val_loss: 0.3819 - val_accuracy: 0.0794
Epoch 5/10
34/34 [==============================] - 440s 13s/step - loss: 0.3607 - accuracy: 0.1017 - val_loss: 0.3466 - val_accuracy: 0.0794
Epoch 6/10
34/34 [==============================] - 445s 13s/step - loss: 0.3557 - accuracy: 0.1017 - val_loss: 0.3494 - val_accuracy: 0.0794
Epoch 7/10
34/34 [==============================] - 440s 13s/step - loss: 0.3596 - accuracy: 0.1017 - val_loss: 0.3491 - val_accuracy: 0.0794
Epoch 8/

In [3]:
from sklearn.metrics import classification_report, multilabel_confusion_matrix, f1_score, accuracy_score
import numpy as np

# Flatten the predictions and true labels for evaluation purposes if necessary
y_true = y_test_binarized  
y_pred = (y_pred_proba > 0.5).astype(int)

# Print the classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, zero_division=0))

# Print the confusion matrix for each class
print("Confusion Matrix for each class:")
cm = multilabel_confusion_matrix(y_true, y_pred)
for i, matrix in enumerate(cm):
    print(f"Class {i}:")
    print(matrix)

# Print the F1 score
f1 = f1_score(y_true, y_pred, average='weighted')  # Use 'weighted' for imbalanced classes
print("F1 Score: {:.2f}".format(f1))

# Print the accuracy
accuracy = accuracy_score(y_true, y_pred)
print("Accuracy: {:.2f}".format(accuracy))


Classification Report:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92       131
           1       0.74      1.00      0.85       113
           2       1.00      1.00      1.00       153
           3       0.65      1.00      0.79       100

   micro avg       0.81      1.00      0.90       497
   macro avg       0.81      1.00      0.89       497
weighted avg       0.83      1.00      0.90       497
 samples avg       0.81      1.00      0.86       497

Confusion Matrix for each class:
Class 0:
[[  0  22]
 [  0 131]]
Class 1:
[[  0  40]
 [  0 113]]
Class 2:
[[  0   0]
 [  0 153]]
Class 3:
[[  0  53]
 [  0 100]]
F1 Score: 0.90
Accuracy: 0.65
